In [1]:
import gc
import time
import xgboost
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from __future__ import division

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))
path = '/Users/shaolongchen/Data Science/Kaggle/Instacart/input/'

aisles.csv
chunk_0.pkl
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
orders_comsum.pkl
previous_products.pkl
prod2vec.pkl
product_period.pkl
product_periods_stat.pkl
products.csv
sample_submission.csv
user_department_products.pkl



/Users/shaolongchen/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))
            
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    '''Create statistical columns, group by [N columns] and compute stats on [N column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       agg_dict: python dictionary

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       {real_column_name: {your_specified_new_column_name : method}}
       agg_dict = {'user_id':{'prod_tot_cnts':'count'},
                   'reordered':{'reorder_tot_cnts_of_this_prod':'sum'},
                   'user_buy_product_times': {'prod_order_once':lambda x: sum(x==1),
                                              'prod_order_more_than_once':lambda x: sum(x==2)}}
       ka_add_stats_features_1_vs_n(train, ['product_id'], agg_dict)
    '''
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    '''Create statistical columns, group by [N columns] and compute stats on [1 column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       target_columns_list: list_like
          column you want to compute stats, need to be a list with only one element
       methods_list: list_like
          methods that you want to use, all methods that supported by groupby in Pandas

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       ka_add_stats_features_n_vs_1(train, group_columns_list=['x0'], target_columns_list=['x10'])
    '''
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new

In [3]:
aisles0 = pd.read_csv(path + "aisles.csv", dtype={'aisle_id': np.uint8, 'aisle':'category'})
aisles = aisles0.copy()
print 'aisles.shape (%s, %s)' % aisles.shape
aisles.head()

aisles.shape (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
departments0 = pd.read_csv(path + "departments.csv", dtype={'department_id':np.uint8, 'department': 'category'})
departments = departments0.copy()
print 'departments.shape  (%s, %s)' % departments.shape
departments.head()

departments.shape  (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
sample_submission0 = pd.read_csv(path + "sample_submission.csv")
sample_submission = sample_submission0.copy()
print 'sample_submission.shape  (%s, %s)' % sample_submission.shape
sample_submission.head()

sample_submission.shape  (75000, 2)


,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [6]:
products0 = pd.read_csv(path + 'products.csv', dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})
products = products0.copy()
print 'products.shape  (%s, %s)' % products.shape
products.head()

products.shape  (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
priors0 = pd.read_csv(path + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
priors = priors0.copy()
print 'priors.shape  (%s, %s)' % priors.shape
priors.head()

priors.shape  (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
train0 = pd.read_csv(path + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
train = train0.copy()
print 'train.shape  (%s, %s)' % train.shape
train.head()

train.shape  (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
orders0 = pd.read_csv(path + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})
orders = orders0.copy()
print 'orders.shape  (%s, %s)' % orders.shape
orders.head()

orders.shape  (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [10]:
# there are 206209 unique users
orders.user_id.describe()

count    3.421083e+06
mean     1.029782e+05
std      5.953372e+04
min      1.000000e+00
25%      5.139400e+04
50%      1.026890e+05
75%      1.543850e+05
max      2.062090e+05
Name: user_id, dtype: float64

In [11]:
# there are 206209 unique users, 75000 of which are in test and 131209 in train
orders.groupby("eval_set")["user_id"].agg(lambda x:len(np.unique(x)))

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

In [12]:
priors_orders= orders.merge(right=priors, how='inner', on='order_id')
print 'priors_orders.shape  (%s, %s)' % priors_orders.shape
priors_orders.head(10)

priors_orders.shape  (32434489, 10)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [13]:
priors_orders['user_buy_product_times'] = priors_orders.groupby(['user_id', 'product_id']).cumcount() + 1


In [14]:
priors_orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1
5,2398795,1,prior,2,3,7,15.0,196,1,1,2
6,2398795,1,prior,2,3,7,15.0,10258,2,0,1
7,2398795,1,prior,2,3,7,15.0,12427,3,1,2
8,2398795,1,prior,2,3,7,15.0,13176,4,0,1
9,2398795,1,prior,2,3,7,15.0,26088,5,1,2


In [15]:
priors_orders.to_csv("priors_orders.csv", index=False)

In [16]:
product_dict = {'user_id':{'product_total_order_times':'count'}, 
            'reordered':{'product_total_reorder_times':'sum'}, 
            'user_buy_product_times': {'total_No_of_user_buy_this_product':lambda x: sum(x==1),
                                        'total_No_of_user_buy_this_product_more_than_twice':lambda x: sum(x==2)}}

In [17]:
product = ka_add_groupby_features_1_vs_n(priors_orders, ['product_id'], product_dict)

add stats features begin ......


/Users/shaolongchen/anaconda/lib/python2.7/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


add stats features end ......
time lapsing 190.399663925 s 



In [18]:
product['product_reorder_probability'] = product.total_No_of_user_buy_this_product_more_than_twice / product.total_No_of_user_buy_this_product
product['product_reorder_ratio'] = product.product_total_reorder_times / product.product_total_order_times
product['product_average_order_times'] = product.product_total_order_times / product.total_No_of_user_buy_this_product

product.head()

,product_id,total_No_of_user_buy_this_product,total_No_of_user_buy_this_product_more_than_twice,product_total_order_times,product_total_reorder_times,product_reorder_probability,product_reorder_ratio,product_average_order_times
0,1,716,276,1852,1136.0,0.385475,0.613391,2.586592
1,2,78,8,90,12.0,0.102564,0.133333,1.153846
2,3,74,36,277,203.0,0.486486,0.732852,3.743243
3,4,182,64,329,147.0,0.351648,0.446809,1.807692
4,5,6,4,15,9.0,0.666667,0.600000,2.500000


In [19]:
product.to_csv("product.csv", index=False)

In [20]:
user_dict1 = {'order_number':{'user_total_order_times':'max'},
             'days_since_prior_order':{'user_sum_days_since_prior_order':'sum', 
                                       'user_mean_days_since_prior_order': 'mean'}}

In [21]:
users1 = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], user_dict1)
users1.head()

add stats features begin ......
add stats features end ......
time lapsing 0.379528999329 s 



,user_id,user_mean_days_since_prior_order,user_sum_days_since_prior_order,user_total_order_times
0,1,19.555555,176.0,10
1,2,15.230769,198.0,14
2,3,12.090909,133.0,12
3,4,13.750000,55.0,5
4,5,13.333333,40.0,4


In [22]:
user_dict2 = {'reordered':
              {'user_reorder_ratio': 
               lambda x: sum(priors_orders.loc[x.index,'reordered']==1)/
                         sum(priors_orders.loc[x.index,'order_number'] > 1)},
              'product_id':{'user_total_ordered_products':'count', 
                            'user_distinct_ordered_products': lambda x: x.nunique()}}

In [23]:
users2 = ka_add_groupby_features_1_vs_n(priors_orders, ['user_id'], user_dict2)
users2.head()

add stats features begin ......
add stats features end ......
time lapsing 517.682731152 s 



,user_id,user_total_ordered_products,user_distinct_ordered_products,user_reorder_ratio
0,1,59,18,0.759259
1,2,195,102,0.510989
2,3,88,33,0.705128
3,4,18,17,0.071429
4,5,37,23,0.538462


In [24]:
users = users1.merge(users2, how='inner')
users.head()

,user_id,user_mean_days_since_prior_order,user_sum_days_since_prior_order,user_total_order_times,user_total_ordered_products,user_distinct_ordered_products,user_reorder_ratio
0,1,19.555555,176.0,10,59,18,0.759259
1,2,15.230769,198.0,14,195,102,0.510989
2,3,12.090909,133.0,12,88,33,0.705128
3,4,13.750000,55.0,5,18,17,0.071429
4,5,13.333333,40.0,4,37,23,0.538462


In [25]:
users['user_average_basket'] = users.user_total_ordered_products / users.user_total_order_times
users.head()

,user_id,user_mean_days_since_prior_order,user_sum_days_since_prior_order,user_total_order_times,user_total_ordered_products,user_distinct_ordered_products,user_reorder_ratio,user_average_basket
0,1,19.555555,176.0,10,59,18,0.759259,5.900000
1,2,15.230769,198.0,14,195,102,0.510989,13.928571
2,3,12.090909,133.0,12,88,33,0.705128,7.333333
3,4,13.750000,55.0,5,18,17,0.071429,3.600000
4,5,13.333333,40.0,4,37,23,0.538462,9.250000


In [26]:
users3 = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]
users3.head()

,user_id,order_id,eval_set,days_since_prior_order
10,1,1187899,train,14.0
25,2,1492625,train,30.0
38,3,2774568,test,11.0
44,4,329954,test,30.0
49,5,2196797,train,6.0


In [27]:
users3.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)
users3.head()

,user_id,order_id,eval_set,time_since_last_order
10,1,1187899,train,14.0
25,2,1492625,train,30.0
38,3,2774568,test,11.0
44,4,329954,test,30.0
49,5,2196797,train,6.0


In [28]:
users = users.merge(users3, how='inner')
users.head()

,user_id,user_mean_days_since_prior_order,user_sum_days_since_prior_order,user_total_order_times,user_total_ordered_products,user_distinct_ordered_products,user_reorder_ratio,user_average_basket,order_id,eval_set,time_since_last_order
0,1,19.555555,176.0,10,59,18,0.759259,5.900000,1187899,train,14.0
1,2,15.230769,198.0,14,195,102,0.510989,13.928571,1492625,train,30.0
2,3,12.090909,133.0,12,88,33,0.705128,7.333333,2774568,test,11.0
3,4,13.750000,55.0,5,18,17,0.071429,3.600000,329954,test,30.0
4,5,13.333333,40.0,4,37,23,0.538462,9.250000,2196797,train,6.0


In [29]:
users.to_csv("users.csv", index=False)

In [30]:
agg_dict_4 = {'order_number':{'user_buy_this_product_total_times': 'count', 
                              'user_buy_this_product_first_order_number': 'min', 
                              'user_buy_this_product_last_order_number':'max'}, 
              'add_to_cart_order':{'user_buy_this_product_median_cart_position': 'median'}}

In [31]:
feature = ka_add_groupby_features_1_vs_n(priors_orders, ['user_id', 'product_id'], agg_dict_4)
feature.head()

add stats features begin ......
add stats features end ......
time lapsing 45.8045480251 s 



,user_id,product_id,user_buy_this_product_median_cart_position,user_buy_this_product_last_order_number,user_buy_this_product_total_times,user_buy_this_product_first_order_number
0,1,196,1.0,10,10,1
1,1,10258,3.0,10,9,2
2,1,10326,5.0,5,1,5
3,1,12427,2.5,10,10,1
4,1,13032,6.0,10,3,2


In [32]:
data = feature.merge(product, how='inner', on='product_id').merge(users, how='inner', on='user_id')
data.head()

,user_id,product_id,user_buy_this_product_median_cart_position,user_buy_this_product_last_order_number,user_buy_this_product_total_times,user_buy_this_product_first_order_number,total_No_of_user_buy_this_product,total_No_of_user_buy_this_product_more_than_twice,product_total_order_times,product_total_reorder_times,...,user_mean_days_since_prior_order,user_sum_days_since_prior_order,user_total_order_times,user_total_ordered_products,user_distinct_ordered_products,user_reorder_ratio,user_average_basket,order_id,eval_set,time_since_last_order
0,1,196,1.0,10,10,1,8000,4660,35791,27791.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
1,1,10258,3.0,10,9,2,557,308,1946,1389.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
2,1,10326,5.0,5,1,5,1923,1003,5526,3603.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
3,1,12427,2.5,10,10,1,1679,889,6476,4797.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
4,1,13032,6.0,10,3,2,1286,617,3751,2465.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0


In [33]:
data['user_buy_this_product_order_rate'] = data.user_buy_this_product_total_times / data.user_total_order_times
data['user_buy_this_product_since_last_order'] = data.user_total_order_times - data.user_buy_this_product_last_order_number
data['user_buy_this_product_order_rate_since_first_order'] = data.user_buy_this_product_total_times / (data.user_total_order_times - data.user_buy_this_product_first_order_number + 1)
print data.shape
data.head()

(13307953, 26)


,user_id,product_id,user_buy_this_product_median_cart_position,user_buy_this_product_last_order_number,user_buy_this_product_total_times,user_buy_this_product_first_order_number,total_No_of_user_buy_this_product,total_No_of_user_buy_this_product_more_than_twice,product_total_order_times,product_total_reorder_times,...,user_total_ordered_products,user_distinct_ordered_products,user_reorder_ratio,user_average_basket,order_id,eval_set,time_since_last_order,user_buy_this_product_order_rate,user_buy_this_product_since_last_order,user_buy_this_product_order_rate_since_first_order
0,1,196,1.0,10,10,1,8000,4660,35791,27791.0,...,59,18,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000
1,1,10258,3.0,10,9,2,557,308,1946,1389.0,...,59,18,0.759259,5.9,1187899,train,14.0,0.9,0,1.000000
2,1,10326,5.0,5,1,5,1923,1003,5526,3603.0,...,59,18,0.759259,5.9,1187899,train,14.0,0.1,5,0.166667
3,1,12427,2.5,10,10,1,1679,889,6476,4797.0,...,59,18,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000
4,1,13032,6.0,10,3,2,1286,617,3751,2465.0,...,59,18,0.759259,5.9,1187899,train,14.0,0.3,0,0.333333


In [34]:
# add user_id to train set
train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
print train.shape
train.head()

(1384617, 5)


,order_id,product_id,add_to_cart_order,reordered,user_id
0,1,49302,1,1,112108
1,1,11109,2,1,112108
2,1,10246,3,0,112108
3,1,49683,4,0,112108
4,1,43633,5,1,112108


In [35]:
data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')
print data.shape
data.head()

(13307953, 27)


,user_id,product_id,user_buy_this_product_median_cart_position,user_buy_this_product_last_order_number,user_buy_this_product_total_times,user_buy_this_product_first_order_number,total_No_of_user_buy_this_product,total_No_of_user_buy_this_product_more_than_twice,product_total_order_times,product_total_reorder_times,...,user_distinct_ordered_products,user_reorder_ratio,user_average_basket,order_id,eval_set,time_since_last_order,user_buy_this_product_order_rate,user_buy_this_product_since_last_order,user_buy_this_product_order_rate_since_first_order,reordered
0,1,196,1.0,10,10,1,8000,4660,35791,27791.0,...,18,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000,1.0
1,1,10258,3.0,10,9,2,557,308,1946,1389.0,...,18,0.759259,5.9,1187899,train,14.0,0.9,0,1.000000,1.0
2,1,10326,5.0,5,1,5,1923,1003,5526,3603.0,...,18,0.759259,5.9,1187899,train,14.0,0.1,5,0.166667,NaN
3,1,12427,2.5,10,10,1,1679,889,6476,4797.0,...,18,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000,NaN
4,1,13032,6.0,10,3,2,1286,617,3751,2465.0,...,18,0.759259,5.9,1187899,train,14.0,0.3,0,0.333333,1.0


In [ ]:
data.to_csv("data.csv", index=False)
print 'Done'